![imagen](data/foto1.png)

### CAPAS DE PREPROCESADO DE KERAS

Igual que sklearn tenemos sus "transformers" y difernentes funciones para poder procesar los datos, para luego, si queremos incluirlos en un pipeline, en Keras existen "capas" de preprocesamiento que podemos incluir en el modelo de forma análoga

### El problema y el dataset

En este caso nuestro problema será clasificar una serie de fármacos a partir de features numéricas y features de texto, que tendremos que convertir para poder utilizar.

Vamos a trabajar con un dataset que hemos empleado antes con reviews de medicinas, pero en el que no utilizamos los campos de texto. Sobre él vamos a hacer el preprocesado usando esas capas equivalentes a lo que ya hemos empleado con sklearn. Eso nos permitirá introducir las capas de Embedding y de ahí a revisar un ejemplo de clasificación con texto en lenguaje natural que ahora haremos con modelos DL.

Primero, las importaciones y cargar el dataset, y echar un "vistazo":

In [ ]:
import random as rm
import re

import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [2]:
df = pd.read_csv("./data/pharma_full.csv")
df.head()

,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,Sales,Production
0,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ...",318440,398.0
1,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest...",888949,909.0
2,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...,264077,465.0
3,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...,542110,602.0
4,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above,83761,124.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3107 entries, 0 to 3106
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   urlDrugName        3107 non-null   object 
 1   rating             3107 non-null   int64  
 2   effectiveness      3107 non-null   object 
 3   sideEffects        3107 non-null   object 
 4   condition          3106 non-null   object 
 5   benefitsReview     3089 non-null   object 
 6   sideEffectsReview  3032 non-null   object 
 7   commentsReview     3095 non-null   object 
 8   Sales              3107 non-null   int64  
 9   Production         3107 non-null   float64
dtypes: float64(1), int64(2), object(7)
memory usage: 242.9+ KB


Las prepararemos un poco para que podamos emplear todos los tipos de capas

Los missings seguiremos tratándolos, por ahora, a parte

In [4]:
df_clean = df.fillna("No Value")

In [5]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3107 entries, 0 to 3106
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   urlDrugName        3107 non-null   object 
 1   rating             3107 non-null   int64  
 2   effectiveness      3107 non-null   object 
 3   sideEffects        3107 non-null   object 
 4   condition          3107 non-null   object 
 5   benefitsReview     3107 non-null   object 
 6   sideEffectsReview  3107 non-null   object 
 7   commentsReview     3107 non-null   object 
 8   Sales              3107 non-null   int64  
 9   Production         3107 non-null   float64
dtypes: float64(1), int64(2), object(7)
memory usage: 242.9+ KB


Claramente el target es "rating"

In [6]:
target = "rating"

### Split train/test

In [7]:
train_set, test_set  = train_test_split(df_clean, test_size=0.2, random_state=42)

In [8]:
train_set.shape

(2485, 10)

In [9]:
test_set.shape

(622, 10)

### Mini-EDA

Del miniEDA sólo vamos a hacer el análisis del target, nos vamos a quedar y a tratar todas las variables como features.

In [10]:
train_set[target].value_counts()

rating
10    591
8     464
9     370
7     267
1     243
5     132
3     123
6     119
4      89
2      87
Name: count, dtype: int64

Es un dataset `desbalanceado` en el target. Luego veremos como equilibrarlo dentro de lo posible sin aplicar SMOTE ni undersampling (con campos con texto además tiene aún menos utilidad, en general)

Como ya sabrás a keras le gusta que las clases empiecen en cero así que lo tratamos rápido:

In [11]:
# forzar que las clases empiecen en cero (porque keras lo necesita)
train_set[target] = train_set[target] - 1
test_set[target] = test_set[target] - 1

### Tratamiento de Features

Vamos a hacer lo siguiente:  
1- Crear features nuevas (conteos de palabras por campo de texto libre)  
2- Escalado de las features numéricas  
3- Conversión de las categóricas  
4- Vectorización de las features de texto  

#### #1 Creación de features nuevas

Existen varios campos con texto libre y en breve los vectorizaremos pero antes puede ser interesante crear una serie de features nuevas que cuenten el número aproximado de palabras que hay en cada uno de esos campos. Estos campos son básicamente lo que tienen en su nombre la palabra "Review"

In [12]:
# word-count
for col in [col for col in train_set.columns if "Review" in col]:
    train_set[col + "_wc"] = train_set[col].apply(lambda value: len(value.split()))
    test_set[col + "_wc"] = test_set[col].apply(lambda value: len(value.split()))


In [13]:
train_set

,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,Sales,Production,benefitsReview_wc,sideEffectsReview_wc,commentsReview_wc
2686,zoloft,9,Highly Effective,No Side Effects,anxiety,becoming a normal functioning individual. it h...,I had no real side effects unless you consider...,taken once daily.,135166,335.0,62,15,3
1192,adipex-p,9,Highly Effective,Mild Side Effects,to lose 30 pounds,When BMI rating is over 27 its time to get tha...,jittery.. sometimes euphoric.. tired but cant ...,last results 3 months on adipex-p lost 30 poun...,739564,939.0,163,10,248
2738,tri-luma,7,Moderately Effective,Severe Side Effects,hyperpigmentation,lightening of acne scars,"redness, dryness, breakdown of skin",this topical compound was used to reduce hyper...,683247,843.0,4,5,42
2780,yasmin,2,Highly Effective,Moderate Side Effects,birth control,There was no chance of pregancy because i lost...,no libido what so ever. My breasts hurt like n...,I was given this to prevent pregnancy and i am...,222295,282.0,22,50,27
1735,requip,7,Considerably Effective,No Side Effects,restless leg syndrome,I took this medication to treat restless leg s...,"I did need to take more as time went on, and I...",I took one milligram at night to treat the sym...,344748,505.0,93,69,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3092,zyrtec,9,Highly Effective,Mild Side Effects,atopic eczema,The drug was very effective in counteracting s...,I felt slightly tired after taking Zyrtec. How...,This is a non-prescription drug and I took it ...,156028,356.0,74,54,51
1095,neurontin,0,Ineffective,Severe Side Effects,all over and various type pain,No Value,"Nausea, extreme sleepiness,severe headache, ji...",Day one I felt negative side effects after tak...,317886,337.0,2,10,120
1130,wellbutrin,6,Moderately Effective,Mild Side Effects,depression,"I experienced less depression, increased libid...",I have a dry mouth and eyes. Drug has decrease...,"Prescribed at 300 mg per day, have taken as mu...",556187,696.0,15,15,24
1294,wellbutrin,2,Considerably Effective,Moderate Side Effects,stop smoking,"The treatment was effective in reducing my ""ap...","After a while, I began to notice that NOTHING ...",I was taking the Bupropion as an aide to break...,728338,789.0,44,105,92


***

#### Escalado/Normalización

Lo primero es quedarnos con las columnas numéricas

In [14]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2485 entries, 2686 to 860
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   urlDrugName           2485 non-null   object 
 1   rating                2485 non-null   int64  
 2   effectiveness         2485 non-null   object 
 3   sideEffects           2485 non-null   object 
 4   condition             2485 non-null   object 
 5   benefitsReview        2485 non-null   object 
 6   sideEffectsReview     2485 non-null   object 
 7   commentsReview        2485 non-null   object 
 8   Sales                 2485 non-null   int64  
 9   Production            2485 non-null   float64
 10  benefitsReview_wc     2485 non-null   int64  
 11  sideEffectsReview_wc  2485 non-null   int64  
 12  commentsReview_wc     2485 non-null   int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 271.8+ KB


Son claramente las que no son tipo `object`

In [15]:
numericas = [col for col in train_set.columns if train_set[col].dtype != "object" and col != target]
numericas

['Sales',
 'Production',
 'benefitsReview_wc',
 'sideEffectsReview_wc',
 'commentsReview_wc']

Y ahora las vamos a normalizar usando keras (en vez de usar sklearn), y además vamos a entrenar un modelo baseline solo con estas features la numéricas.

### Normalization Layer

In [16]:
X_num = train_set[numericas].copy()
y_num = train_set[target]

Probemos primero sin normalizar, creando un diccionario para compensar las clases:


In [17]:
# extra code – ensures reproducibility
tf.random.set_seed(42)  

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
    ])

earlyS = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True, monitor="val_acc")

model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              metrics =["acc"]
              )


In [18]:
history = model.fit(X_num, y_num, validation_split=0.2, epochs=300, callbacks=earlyS)

Epoch 1/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - acc: 0.2007 - loss: 22270644224.0000 - val_acc: 0.2233 - val_loss: 2.2798
Epoch 2/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.2414 - loss: 2.2711 - val_acc: 0.2233 - val_loss: 2.2572
Epoch 3/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.2414 - loss: 2.2489 - val_acc: 0.2233 - val_loss: 2.2375
Epoch 4/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2414 - loss: 2.2296 - val_acc: 0.2233 - val_loss: 2.2204
Epoch 5/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2414 - loss: 2.2128 - val_acc: 0.2233 - val_loss: 2.2056
Epoch 6/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2414 - loss: 2.1983 - val_acc: 0.2233 - val_loss: 2.1928
Epoch 7/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2414 - loss: 2.1857 - val_acc: 0.2233 - val_loss: 2.1817
Epoch 8/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2414 - loss: 2.1749 - val_acc: 0.2233 - val_loss: 2.1721
Epoch 9/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - a

In [19]:
model.evaluate(test_set[numericas], test_set[target])

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2589 - loss: 2.2804 


[2.279801607131958, 0.24276527762413025]

La cosa está fácil... Vamos con la normalización

In [20]:
tf.random.set_seed(42) 

# Realmente hace una estandardization. Necestiamos hacer un "fit" previo como con el StandardScaler
norm_layer = tf.keras.layers.Normalization() 
 
model = tf.keras.models.Sequential([
    norm_layer,
    tf.keras.layers.Dense(100, activation = "relu"),
    tf.keras.layers.Dense(10, activation = "softmax")
])

model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              metrics =["acc"])

# esto es similar al fit. Depende la versión de tensorflow puede o no trabajar directamente con DataFrames
norm_layer.adapt(X_num.to_numpy())

history = model.fit(X_num, y_num, validation_split=0.2, epochs=300, callbacks=earlyS)

Epoch 1/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - acc: 0.1087 - loss: 2.3176 - val_acc: 0.1670 - val_loss: 2.2228
Epoch 2/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.1637 - loss: 2.2338 - val_acc: 0.2093 - val_loss: 2.1665
Epoch 3/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2097 - loss: 2.1734 - val_acc: 0.2113 - val_loss: 2.1263
Epoch 4/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2393 - loss: 2.1302 - val_acc: 0.2435 - val_loss: 2.0977
Epoch 5/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2513 - loss: 2.0995 - val_acc: 0.2676 - val_loss: 2.0769
Epoch 6/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2577 - loss: 2.0770 - val_acc: 0.2757 - val_loss: 2.0614
Epoch 7/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.2624 - loss: 2.0600 - val_acc: 0.2736 - val_loss: 2.0493
Epoch 8/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.2681 - loss: 2.0465 - val_acc: 0.2716 - val_loss: 2.0394
Epoch 9/300
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.2758

In [21]:
model.evaluate(test_set[numericas], test_set[target])

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.7283 - loss: 0.8356


[0.8436590433120728, 0.7266880869865417]

In [22]:
y_pred = [np.argmax(pred) for pred in model.predict(test_set[numericas])]

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [24]:
print(classification_report(test_set[target], y_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89        62
           1       0.00      0.00      0.00        16
           2       0.73      0.83      0.78        23
           3       0.50      0.28      0.36        18
           4       0.59      0.63      0.61        27
           5       0.00      0.00      0.00        38
           6       0.21      0.13      0.16        83
           7       0.53      0.99      0.69        94
           8       0.99      0.85      0.92       110
           9       0.96      1.00      0.98       151

    accuracy                           0.73       622
   macro avg       0.53      0.57      0.54       622
weighted avg       0.66      0.73      0.68       622



c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Bien, pues ya tenemos baseline y una forma sencilla de incorporar la normalización a nuestro modelo, vamos con el resto de transformaciones, pero ya en la siguiente sesión.

***

### Transformación de variables categóricas

Vamos a transformar las categóricas y lo vamos a hacer con un "ordinal encoder" por un lado y con un "onehot" por otro, eso sí, usando las capas de Keras, y cuáles son categóricas:

In [25]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2485 entries, 2686 to 860
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   urlDrugName           2485 non-null   object 
 1   rating                2485 non-null   int64  
 2   effectiveness         2485 non-null   object 
 3   sideEffects           2485 non-null   object 
 4   condition             2485 non-null   object 
 5   benefitsReview        2485 non-null   object 
 6   sideEffectsReview     2485 non-null   object 
 7   commentsReview        2485 non-null   object 
 8   Sales                 2485 non-null   int64  
 9   Production            2485 non-null   float64
 10  benefitsReview_wc     2485 non-null   int64  
 11  sideEffectsReview_wc  2485 non-null   int64  
 12  commentsReview_wc     2485 non-null   int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 271.8+ KB


Tiene pinta de que todas las que son objetc que no son Review

In [26]:
# definir las categóricas
categoricals = [col for col in train_set.columns if "Review" not in col and train_set[col].dtype == "object"]
categoricals

['urlDrugName', 'effectiveness', 'sideEffects', 'condition']

Hagamos un check de cardinalidad:

In [27]:
for col in categoricals:
    print(f"Para <{col}>: {train_set[col].nunique()}")

Para <urlDrugName>: 463
Para <effectiveness>: 5
Para <sideEffects>: 5
Para <condition>: 1203


In [28]:
train_set.condition.value_counts()

condition
depression                        182
acne                              135
anxiety                            51
insomnia                           41
birth control                      34
                                 ... 
major sinus infection               1
bipolar/raciness/insomnia           1
anxiety/insomnia/                   1
infected site of oreal surgery      1
all over and various type pain      1
Name: count, Length: 1203, dtype: int64

Tenemos que quitar la url y condition. Esta además tendría que tener un tratamiento particular (entre vectorización y compresión de categorías, que no vamos a hacer por tiempo y espacio)

In [29]:
categoricals.remove("urlDrugName")
categoricals.remove("condition")
categoricals

['effectiveness', 'sideEffects']

### Ordinal Encoding layer (aka StringLookup)

La forma de hacer el categórical encoding es unsar una capa de stringlookup que básicamente puede hacer ella el mapeo o se le puede dar (como en un ordinal de sklearn)

Vamos a añadir las categóricas de dos formas, pero primero vamos a hacerles su ordinal encoding. El único pero es que hay que hacerlo feature a feature

In [30]:
X_train = train_set[numericas + categoricals].copy()
#X_train[categoricals] = ordinalEncoding(train_set[categoricals])

X_test = test_set[numericas + categoricals].copy()
#X_test[categoricals] = ordinalEncoding(test_set[categoricals])

In [31]:
tf.random.set_seed(42)

normalization_layer = tf.keras.layers.Normalization()

# no le damos valor, pero aquí el argumento vocabulary permite pasar una lista,
# una tupla, un array, un tensor con los strings ordenados de forma que el primero recibira la clase 0, etc
ordinalEncoding_layers = [tf.keras.layers.StringLookup() for cat in categoricals]                                               

# ordinalEncoding.adapt(train_set[categoricals]) # Al igual que la capa de normalización hay que hacerle su fit en este caso el método adapt
concat_layer = tf.keras.layers.Concatenate(axis=-1)
hidden_layer1 = tf.keras.layers.Dense(100, activation="relu")
output_layer = tf.keras.layers.Dense(10, activation="softmax")

normalization_layer.adapt(train_set[numericas].to_numpy())
# truco de programación para hacerlo en un única linea
_ = [ordinalEncoding_layers[indice].adapt(train_set[col]) for indice,col in enumerate(categoricals)]

earlyS = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
input_num = tf.keras.layers.Input(shape=(train_set[numericas].shape[1],))
input_cats = [tf.keras.layers.Input(shape=(train_set[[col]].shape[1],), dtype = tf.string) for col in categoricals] # Importante decirle que entra una string
normalized = normalization_layer(input_num)
encoded = [ordinalEncoding_layers[i](input_cats[i]) for i in range(len(ordinalEncoding_layers))]
concat = concat_layer([normalized] + encoded)
hidden1 = hidden_layer1(concat)
output = output_layer(hidden1)

model = tf.keras.Model(inputs=[input_num] + input_cats, outputs=[output])

In [32]:
model.compile(loss="sparse_categorical_crossentropy", \
              optimizer=tf.keras.optimizers.SGD(learning_rate=2e-1),\
              metrics =["acc"])


In [33]:
salida = tf.keras.Model(inputs=input_cats, outputs=encoded)
salida([X_train[col] for col in categoricals])

[<tf.Tensor: shape=(2485, 1), dtype=int64, numpy=
 array([[1],
        [1],
        [3],
        ...,
        [3],
        [2],
        [1]], dtype=int64)>,
 <tf.Tensor: shape=(2485, 1), dtype=int64, numpy=
 array([[2],
        [1],
        [4],
        ...,
        [1],
        [3],
        [5]], dtype=int64)>]

In [34]:
X_train[[categoricals[0]]]

,effectiveness
2686,Highly Effective
1192,Highly Effective
2738,Moderately Effective
2780,Highly Effective
1735,Considerably Effective
...,...
3092,Highly Effective
1095,Ineffective
1130,Moderately Effective
1294,Considerably Effective


In [35]:
entradas = [X_train[numericas]] + [X_train[col] for col in categoricals]

In [36]:
history = model.fit(entradas, y_num, validation_split=0.2, epochs=100, callbacks= earlyS)

Epoch 1/100
 1/63 ━━━━━━━━━━━━━━━━━━━━ 14s 236ms/step - acc: 0.1875 - loss: 2.5168

c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_7', 'keras_tensor_8', 'keras_tensor_9']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.2683 - loss: 2.1325 - val_acc: 0.3139 - val_loss: 1.8725
Epoch 2/100
57/63 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - acc: 0.4043 - loss: 1.7092

c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_7', 'keras_tensor_8', 'keras_tensor_9']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.4051 - loss: 1.7058 - val_acc: 0.3622 - val_loss: 1.6759
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.4336 - loss: 1.5413 - val_acc: 0.3843 - val_loss: 1.5912
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.4479 - loss: 1.4548 - val_acc: 0.4064 - val_loss: 1.5400
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.4580 - loss: 1.3973 - val_acc: 0.4044 - val_loss: 1.5046
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.4729 - loss: 1.3540 - val_acc: 0.4185 - val_loss: 1.4715
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.4863 - loss: 1.3159 - val_acc: 0.4185 - val_loss: 1.4418
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.4885 - loss: 1.2836 - val_acc: 0.4266 - val_loss: 1.4020
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.4875 - loss: 1.2528 - val_acc: 0.4245 - val_loss: 1.3865
Epoch 10/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.4927 - loss: 1.

In [37]:
entrada_test = [X_test[numericas]] + [X_test[col] for col in categoricals]
model.evaluate(entrada_test, test_set[target])

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.9234 - loss: 0.3104 


[0.33659836649894714, 0.9067524075508118]

Algo hemos mejorado con esta categorización, ¿no?

In [38]:
y_pred  = [np.argmax(pred) for pred in model.predict(entrada_test)]

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_7', 'keras_tensor_8', 'keras_tensor_9']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


In [39]:
print(classification_report(test_set[target], y_pred))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93        62
           1       1.00      0.38      0.55        16
           2       0.00      0.00      0.00        23
           3       0.33      0.83      0.48        18
           4       0.69      0.67      0.68        27
           5       0.91      0.76      0.83        38
           6       0.95      0.95      0.95        83
           7       0.98      1.00      0.99        94
           8       1.00      1.00      1.00       110
           9       1.00      1.00      1.00       151

    accuracy                           0.91       622
   macro avg       0.77      0.76      0.74       622
weighted avg       0.90      0.91      0.90       622



#### Onehotencoding

Pero podemos hacer el onehot encoding de una vez configurando la StringLookup layer debidamente.  Además ahora usaremos la functional API para incluir la capa dentro del modelo (y no tener que hacer la conversión por fuera)

In [40]:
tf.random.set_seed(42)

normalization_layer = tf.keras.layers.Normalization()
# no le damos valor, pero aquí el argumento vocabulary permite pasar una lista,
# una tupla, un array, un tensor con los strings ordenados de forma que el primero recibira la clase 0, etc
ordinalEncoding_layers = [tf.keras.layers.StringLookup(output_mode="one_hot") for cat in categoricals]                                              

# ordinalEncoding.adapt(train_set[categoricals]) # Al igual que la capa de normalización hay que hacerle su fit en este caso el método adapt
concat_layer = tf.keras.layers.Concatenate(axis = -1)
hidden_layer1 = tf.keras.layers.Dense(100, activation="relu")
output_layer = tf.keras.layers.Dense(10, activation="softmax")

normalization_layer.adapt(train_set[numericas].to_numpy())
_ = [ordinalEncoding_layers[indice].adapt(train_set[col]) for indice,col in enumerate(categoricals)]

earlyS = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights= True)
input_num = tf.keras.layers.Input(shape=(train_set[numericas].shape[1],))
input_cats = [tf.keras.layers.Input(shape=(train_set[[col]].shape[1],), dtype = tf.string) for col in categoricals] # Importante decirle que entra una string
normalized = normalization_layer(input_num)
encoded = [ordinalEncoding_layers[i](input_cats[i]) for i in range(len(ordinalEncoding_layers))]
concat = concat_layer([normalized] + encoded)
hidden1 = hidden_layer1(concat)
output = output_layer(hidden1)

model = tf.keras.Model(inputs=[input_num] + input_cats, outputs=[output])

In [41]:
salida = tf.keras.Model(inputs = input_cats, outputs = encoded)
salida([X_train[col] for col in categoricals])

[<tf.Tensor: shape=(2485, 6), dtype=int64, numpy=
 array([[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        ...,
        [0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0]], dtype=int64)>,
 <tf.Tensor: shape=(2485, 6), dtype=int64, numpy=
 array([[0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        ...,
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1]], dtype=int64)>]

In [42]:
X_train[[categoricals[1]]]

,sideEffects
2686,No Side Effects
1192,Mild Side Effects
2738,Severe Side Effects
2780,Moderate Side Effects
1735,No Side Effects
...,...
3092,Mild Side Effects
1095,Severe Side Effects
1130,Mild Side Effects
1294,Moderate Side Effects


In [43]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=tf.keras.optimizers.SGD(learning_rate=2e-1),
              metrics =["acc"])


In [44]:
entradas = [X_train[numericas]] + [X_train[col] for col in categoricals]

In [45]:
history = model.fit(entradas, y_num, validation_split=0.2, epochs=100, callbacks=earlyS)

Epoch 1/100


c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_16', 'keras_tensor_17', 'keras_tensor_18']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.3311 - loss: 1.9715 - val_acc: 0.3823 - val_loss: 1.6460
Epoch 2/100
 2/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.4531 - loss: 1.51400

c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_16', 'keras_tensor_17', 'keras_tensor_18']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.4508 - loss: 1.5144 - val_acc: 0.3903 - val_loss: 1.5107
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.4699 - loss: 1.3812 - val_acc: 0.4125 - val_loss: 1.4524
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.4853 - loss: 1.3160 - val_acc: 0.4245 - val_loss: 1.4177
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.4975 - loss: 1.2761 - val_acc: 0.4427 - val_loss: 1.3946
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.5033 - loss: 1.2466 - val_acc: 0.4427 - val_loss: 1.3734
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.5084 - loss: 1.2196 - val_acc: 0.4487 - val_loss: 1.3537
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.5147 - loss: 1.1944 - val_acc: 0.4588 - val_loss: 1.3347
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.5249 - loss: 1.1693 - val_acc: 0.4588 - val_loss: 1.3188
Epoch 10/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.5302 - loss: 1.

In [46]:
model.evaluate(entrada_test,test_set[target])

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.9421 - loss: 0.2209


[0.23860207200050354, 0.9340835809707642]

Un poco mejor que con el ordinal, pero sin pasarse y además hay que ver el impacto del desbalanceo

In [47]:
y_pred  = [np.argmax(pred) for pred in model.predict(entrada_test)]

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_16', 'keras_tensor_17', 'keras_tensor_18']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


In [48]:
print(classification_report(test_set[target], y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        62
           1       1.00      0.38      0.55        16
           2       0.61      0.83      0.70        23
           3       0.70      0.39      0.50        18
           4       0.57      1.00      0.73        27
           5       0.89      0.66      0.76        38
           6       0.98      0.96      0.97        83
           7       1.00      1.00      1.00        94
           8       1.00      1.00      1.00       110
           9       1.00      1.00      1.00       151

    accuracy                           0.93       622
   macro avg       0.87      0.82      0.82       622
weighted avg       0.95      0.93      0.93       622



Se nos va de nuevo a la clase mayoritaria. En la siguiente sesión añadiremos la vectorización de las features de texto y te enseñare a configurar el class_weight para aliviar (no siempre mucho) el efecto del desbalanceo

***

Primero vamos a mostrar rápidamente como funciona la capa de vectorizacion de texto de keras y luego aplicaremos el proceso completo (recuerda que hay que limpiar el texto y tokenizarlo antes de hacer su vectorización). Vamos con la(s) capa(s) de vectorización

## Capas de Vectorizacion

El equivalente al CountVectorizer y al TfidfVectorizer de sklearn es la capa TextVectorization

In [49]:
 # output_mode="count" -> Countvectorizer
text_vec_layer_count = tf.keras.layers.TextVectorization(output_mode="count")

text_vec_layer_count.adapt(train_set[["sideEffectsReview"]])

In [50]:
text_vec_layer_count.get_vocabulary()[:10]

['[UNK]', 'i', 'the', 'and', 'to', 'a', 'of', 'my', 'it', 'was']

In [51]:
texto = train_set["sideEffectsReview"][2:3].values
print(texto)

['redness, dryness, breakdown of skin']


In [52]:
# output_mode="tf_idf" --> TfIdfVectorizer, existe un tercer modo (el que viene por defecto que veremos un poco más adelante)
text_vec_layer_tfidf = tf.keras.layers.TextVectorization(output_mode="tf_idf")
text_vec_layer_tfidf.adapt(train_set["sideEffectsReview"])

In [54]:
vectors = text_vec_layer_count(train_set["sideEffectsReview"])
vectors

<tf.Tensor: shape=(2485, 6564), dtype=int64, numpy=
array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 8, 3, ..., 0, 0, 0],
       [0, 1, 2, ..., 0, 0, 0]], dtype=int64)>

In [56]:
df_vectors = pd.DataFrame(vectors.numpy(),
                          columns= text_vec_layer_count.get_vocabulary())
df_vectors

,[UNK],i,the,and,to,a,of,my,it,was,...,10142008,1014,1012,100mgthe,100mgs,100mgdoses,100110,1000mg,10000,072009
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,3,2,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,4,7,3,3,0,2,0,2,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,0,3,5,1,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
2481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2482,0,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2483,0,8,3,1,7,3,3,2,1,3,...,0,0,0,0,0,0,0,0,0,0


In [57]:
train_set.iloc[0]["sideEffectsReview"]

'I had no real side effects unless you consider lack of fear or worry one.'

In [58]:
df_vectors.T[df_vectors.T[0] != 0][0]


i           1
of          1
side        1
effects     1
had         1
no          1
or          1
you         1
one         1
lack        1
real        1
unless      1
consider    1
fear        1
worry       1
Name: 0, dtype: int64

#### Vectorizacion

Tendremos que hacer la normalización/limpieza del texto y "tokenizacion"

In [78]:
# descargar las stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lander\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
# Rescatando la que hicimos para la IMDB Reviews

replace_no_space = "(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)"
REPLACE_NO_SPACE = re.compile(replace_no_space)
replace_with_space = "(<br \s*/><br\s*/>)|(\-)|(\/)"
REPLACE_WITH_SPACE = re.compile(replace_with_space)
NO_SPACE = ""
SPACE = " "
dictionary = stopwords.words("english")

def clean(row):
    # Limpio signos y convierto a minúsculas
    dato = REPLACE_NO_SPACE.sub(NO_SPACE, row.lower())
    # Convierto los retornos de carro <br /><br /> en espacios y los guiones ("-")
    dato = REPLACE_WITH_SPACE.sub(SPACE, dato)
    # Quito cualquier link
    dato = " ".join([word for word in dato.split() if "http" not in word])
    # Quito los stopwords
    dato = " ".join([word for word in dato.split(" ") if word not in dictionary])
    return dato


<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Lander\AppData\Local\Temp\ipykernel_8912\2802507076.py:4: SyntaxWarning: invalid escape sequence '\.'
  replace_no_space = "(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)"
C:\Users\Lander\AppData\Local\Temp\ipykernel_8912\2802507076.py:6: SyntaxWarning: invalid escape sequence '\s'
  replace_with_space = "(<br \s*/><br\s*/>)|(\-)|(\/)"


Antes, identificamos las features que son texto natural:


In [61]:
text_feats = [col for col in train_set.columns if "Review" in col and "wc" not in col]
text_feats

['benefitsReview', 'sideEffectsReview', 'commentsReview']


Y ahora aplicamos

In [62]:
for col in text_feats:
    train_set[col] = train_set[col].apply(clean)
    test_set[col] = test_set[col].apply(clean)

In [63]:
train_set

,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,Sales,Production,benefitsReview_wc,sideEffectsReview_wc,commentsReview_wc
2686,zoloft,9,Highly Effective,No Side Effects,anxiety,becoming normal functioning individual helped ...,real side effects unless consider lack fear wo...,taken daily,135166,335.0,62,15,3
1192,adipex-p,9,Highly Effective,Mild Side Effects,to lose 30 pounds,bmi rating time get taken care goes beyond tak...,jittery sometimes euphoric tired cant sleep zone,last results months adipex p lost pounds gaine...,739564,939.0,163,10,248
2738,tri-luma,7,Moderately Effective,Severe Side Effects,hyperpigmentation,lightening acne scars,redness dryness breakdown skin,topical compound used reduce hyperpigmentation...,683247,843.0,4,5,42
2780,yasmin,2,Highly Effective,Moderate Side Effects,birth control,chance pregancy lost percent libido skin clear...,libido ever breasts hurt like got bigger itche...,given prevent pregnancy going i'd rather child...,222295,282.0,22,50,27
1735,requip,7,Considerably Effective,No Side Effects,restless leg syndrome,took medication treat restless leg syndrome ef...,need take time went experienced augmentation r...,took one milligram night treat symtoms restles...,344748,505.0,93,69,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3092,zyrtec,9,Highly Effective,Mild Side Effects,atopic eczema,drug effective counteracting severe itching ca...,felt slightly tired taking zyrtec however seve...,non prescription drug took experiencing severe...,156028,356.0,74,54,51
1095,neurontin,0,Ineffective,Severe Side Effects,all over and various type pain,value,nausea extreme sleepinesssevere headache jitte...,day one felt negative side effects taking seco...,317886,337.0,2,10,120
1130,wellbutrin,6,Moderately Effective,Mild Side Effects,depression,experienced less depression increased libido p...,dry mouth eyes drug decreased effectiveness la...,prescribed mg per day taken much mg per day li...,556187,696.0,15,15,24
1294,wellbutrin,2,Considerably Effective,Moderate Side Effects,stop smoking,treatment effective reducing appetite smoking ...,began notice nothing seemed important also see...,taking bupropion aide break smoking habit also...,728338,789.0,44,105,92


In [64]:
print("\n\n".join(train_set["sideEffectsReview"][0:2].values))

real side effects unless consider lack fear worry one

jittery sometimes euphoric tired cant sleep zone


Es hora de aplicar la "capa" de vectorizacion, pero keras sólo admite una entrada o feature por capa así que hay que crear tres

In [65]:
vectorizer_layers = []
for indice,col in enumerate(text_feats):
    vectorizer_layers.append(tf.keras.layers.TextVectorization(output_mode="count"))
    vectorizer_layers[indice].adapt(train_set[col])

Y ahora creamos el modelo con la API funcional para intregarlo todo

In [66]:
tf.random.set_seed(43)
normalization_layer = tf.keras.layers.Normalization()
# no le damos valor, pero aquí el argumento vocabulary permite pasar una lista,ç
# una tupla, un array, un tensor con los strings ordenados de forma que el primero recibira la clase 0, etc
ordinalEncoding_layers = [tf.keras.layers.StringLookup(output_mode="one_hot") for cat in categoricals] 
                                                 

# ordinalEncoding.adapt(train_set[categoricals]). Al igual que la capa de normalización hay que hacerle su fit en este caso el método adapt
concat_layer = tf.keras.layers.Concatenate(axis = -1)
hidden_layer1 = tf.keras.layers.Dense(100, activation="relu")
output_layer = tf.keras.layers.Dense(10, activation = "softmax")


normalization_layer.adapt(train_set[numericas].to_numpy())
_ = [ordinalEncoding_layers[indice].adapt(train_set[col]) for indice,col in enumerate(categoricals)]

earlyS = tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights= True)
input_num = tf.keras.layers.Input(shape=(train_set[numericas].shape[1],))
input_cats = [tf.keras.layers.Input(shape=(train_set[[col]].shape[1],), dtype = tf.string) for col in categoricals] # Importante decirle que entra una string

# Añadimos las entradas para las capas de vectorizacion
input_vects = [tf.keras.layers.Input(shape = train_set[col].shape[1:], dtype = tf.string) for col in text_feats]
normalized = normalization_layer(input_num)
encoded = [ordinalEncoding_layers[i](input_cats[i]) for i in range(len(ordinalEncoding_layers))]

# Ahora vectorizamos
vectorized = [vectorizer_layers[i](input_vects[i]) for i in range(len(text_feats))]
concat = concat_layer([normalized] + encoded + vectorized)
hidden1 = hidden_layer1(concat)
output = output_layer(hidden1)

model = tf.keras.Model(inputs=[input_num] + input_cats + input_vects, outputs=[output])

In [67]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=tf.keras.optimizers.SGD(learning_rate= 2e-2, momentum = 0.9),
              metrics =["acc"])

entradas = [train_set[numericas]] + [train_set[col] for col in categoricals] \
                + [train_set[col] for col in text_feats]               
history = model.fit(entradas, y_num, validation_split=0.2, epochs=150, callbacks=earlyS)

Epoch 1/150


c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_25', 'keras_tensor_26', 'keras_tensor_27', 'keras_tensor_28', 'keras_tensor_29', 'keras_tensor_30']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


59/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - acc: 0.2587 - loss: 2.1173

c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_25', 'keras_tensor_26', 'keras_tensor_27', 'keras_tensor_28', 'keras_tensor_29', 'keras_tensor_30']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - acc: 0.2642 - loss: 2.1027 - val_acc: 0.3863 - val_loss: 1.7189
Epoch 2/150
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.4937 - loss: 1.3769 - val_acc: 0.4245 - val_loss: 1.5686
Epoch 3/150
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7274 - loss: 0.8940 - val_acc: 0.4346 - val_loss: 1.5979
Epoch 4/150
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8917 - loss: 0.5381 - val_acc: 0.4386 - val_loss: 1.7142
Epoch 5/150
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9601 - loss: 0.3108 - val_acc: 0.4386 - val_loss: 1.7800
Epoch 6/150
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9833 - loss: 0.1890 - val_acc: 0.4145 - val_loss: 1.7910
Epoch 7/150
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.9915 - loss: 0.1221 - val_acc: 0.4205 - val_loss: 1.7939
Epoch 8/150
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9954 - loss: 0.0858 - val_acc: 0.4205 - val_loss: 1.8627
Epoch 9/150
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9987 - loss: 0.

In [68]:
entradas_test = [test_set[numericas]] + [test_set[col] for col in test_set[categoricals]]\
                                         + [test_set[col] for col in text_feats]
model.evaluate(entradas_test, test_set[target])

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.4150 - loss: 1.4902


[1.5267363786697388, 0.41639870405197144]

In [69]:
y_pred = [np.argmax(pred) for pred in model.predict(entradas_test)]

15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  

c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_25', 'keras_tensor_26', 'keras_tensor_27', 'keras_tensor_28', 'keras_tensor_29', 'keras_tensor_30']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [70]:
print(classification_report(test_set[target], y_pred))

              precision    recall  f1-score   support

           0       0.60      0.87      0.71        62
           1       0.00      0.00      0.00        16
           2       0.50      0.09      0.15        23
           3       0.50      0.06      0.10        18
           4       0.00      0.00      0.00        27
           5       0.27      0.08      0.12        38
           6       0.17      0.07      0.10        83
           7       0.26      0.66      0.37        94
           8       0.36      0.04      0.07       110
           9       0.57      0.84      0.68       151

    accuracy                           0.42       622
   macro avg       0.32      0.27      0.23       622
weighted avg       0.37      0.42      0.33       622



Desbalanceado y el recall medio cae. Además son demasiadas features. En la próxima sesión, de caracter teórico veremos una forma más potente de vectorizar texto y para terminar esta veamos como "compensar"

In [71]:
y_num.value_counts(True)

rating
9    0.237827
7    0.186720
8    0.148893
6    0.107445
0    0.097787
4    0.053119
2    0.049497
5    0.047887
3    0.035815
1    0.035010
Name: proportion, dtype: float64

In [72]:
clases = np.array(sorted(y_num.unique()))
clases


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [73]:
dict_weight = {clases[i]:peso for i,peso in enumerate(compute_class_weight("balanced", classes= clases, y=y_num))}
dict_weight


{0: 1.022633744855967,
 1: 2.8563218390804597,
 2: 2.0203252032520327,
 3: 2.792134831460674,
 4: 1.8825757575757576,
 5: 2.088235294117647,
 6: 0.9307116104868914,
 7: 0.5355603448275862,
 8: 0.6716216216216216,
 9: 0.42047377326565144}

In [74]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-1, momentum=0.9, nesterov=True),
              metrics =["acc"])

history = model.fit(entradas, y_num.to_numpy(), validation_split=0.2, epochs=10, callbacks=earlyS, class_weight=dict_weight)


Epoch 1/10


c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_25', 'keras_tensor_26', 'keras_tensor_27', 'keras_tensor_28', 'keras_tensor_29', 'keras_tensor_30']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


59/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.6206 - loss: 1.3136

c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_25', 'keras_tensor_26', 'keras_tensor_27', 'keras_tensor_28', 'keras_tensor_29', 'keras_tensor_30']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - acc: 0.6161 - loss: 1.3191 - val_acc: 0.3863 - val_loss: 1.8014
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8763 - loss: 0.4220 - val_acc: 0.4004 - val_loss: 2.4331
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.9699 - loss: 0.1028 - val_acc: 0.4145 - val_loss: 2.5956
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.9931 - loss: 0.0326 - val_acc: 0.3883 - val_loss: 2.7220
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.9983 - loss: 0.0164 - val_acc: 0.3883 - val_loss: 2.9109
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - acc: 0.9971 - loss: 0.0203 - val_acc: 0.4205 - val_loss: 2.8752
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 1.0000 - loss: 0.0047 - val_acc: 0.4266 - val_loss: 2.8819
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 1.0000 - loss: 0.0025 - val_acc: 0.4266 - val_loss: 2.9055
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 1.0000 - loss: 0.0021 - v

In [75]:

model.evaluate(entradas_test, test_set[target].to_numpy())

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.4269 - loss: 1.6762


[1.7642327547073364, 0.4003215432167053]

In [76]:
y_pred = [np.argmax(pred) for pred in model.predict(entradas_test)]

15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  

c:\Users\Lander\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_25', 'keras_tensor_26', 'keras_tensor_27', 'keras_tensor_28', 'keras_tensor_29', 'keras_tensor_30']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [77]:
print(classification_report(test_set[target], y_pred))

              precision    recall  f1-score   support

           0       0.67      0.69      0.68        62
           1       0.38      0.19      0.25        16
           2       0.22      0.22      0.22        23
           3       0.08      0.11      0.10        18
           4       0.11      0.44      0.18        27
           5       0.31      0.29      0.30        38
           6       0.17      0.06      0.09        83
           7       0.32      0.41      0.36        94
           8       0.42      0.10      0.16       110
           9       0.64      0.78      0.71       151

    accuracy                           0.40       622
   macro avg       0.33      0.33      0.30       622
weighted avg       0.41      0.40      0.38       622

